In [10]:
import os
os.chdir("/Users/isabellawu/Desktop/RA-Bingjing/schoolpre")

# 获取当前工作目录
current_dir = os.getcwd()
print(current_dir)

/Users/isabellawu/Desktop/RA-Bingjing/schoolpre


In [11]:
# setting.py 为项目配置文件

# 配置API服务

HOST = "0.0.0.0"               # IP
PORT = 5000                    # 监听端口


# 配置数据库

DB_CONN = 'redis://:pwd@127.0.0.1:8888/0'


# 配置 ProxyFetcher

PROXY_FETCHER = [
    "freeProxy01",      # 这里是启用的代理抓取方法名，所有fetch方法位于fetcher/proxyFetcher.py
    "freeProxy02",
    "freeProxy03",
    "freeProxy04",
    "freeProxy05",
    "freeProxy06",
    "freeProxy07",
    "freeProxy08",
    "freeProxy09",
    "freeProxy10",
    "freeProxy11"
    # ....
]

In [12]:
# -*- coding:utf-8 -*-
import hashlib
import requests
import json
import pandas as pd
from fake_useragent import UserAgent
import time
import random
import csv



In [13]:

# 获得ip
def get_proxy():
    return requests.get("http://127.0.0.1:5010/get/").json()


def delete_proxy(proxy):
    requests.get("http://127.0.0.1:5010/delete/?proxy={}".format(proxy))


def myhash(data, flag="md5"):  ##加密文本
    """
    hash 函数 默认 md5 可以使用 sha1 等
    :param data: 待加密的文本
    :param flag: md5/sha1/sha256等
    :return:
    """
    if hasattr(hashlib, flag):
        my_hash = getattr(hashlib, flag)()
    else:

        return
    my_hash.update(data.encode("utf8"))
    hash_data = my_hash.hexdigest()
    return hash_data


# 提取参数
def extract_batch_data(school_id):
    url = f'https://static-data.gaokao.cn/www/2.0/school/{school_id}/dic/specialplan.json'

    # 发起 HTTP 请求获取 specialscore.json 的 XHR 响应
    response = requests.get(url)

    # 检查响应状态
    if response.status_code != 200:
        print(f"Failed to retrieve data for school {school_id}")
        return None

    # 将响应的 JSON 数据解析为 Python 对象
    data = json.loads(response.text)

    # 提取 batch 数据
    batch_data = data['data']['newsdata']['batch']

    # 创建空的 DataFrame
    df = pd.DataFrame(columns=['province', 'year', 'data_type', 'batch'])

    # 遍历 batch 数据
    for key, value in batch_data.items():
        province, year, data_type = key.split('_')
        df = pd.concat([df, pd.DataFrame({
            'province': [province] * len(value),
            'year': [year] * len(value),
            'data_type': [data_type] * len(value),
            'batch': value
        })])

    # 重置索引
    df.reset_index(drop=True, inplace=True)

    return df


def get_data(school_id, time_str, local_id, batch_id, type_id, page):
    headers = {
        "authority": "api.zjzw.cn",
        "accept": "application/json, text/plain, */*",
        "accept-language": "zh-CN,zh;q=0.9,en;q=0.8,zh-TW;q=0.7",
        "cache-control": "no-cache",
        "content-type": "application/json",
        "origin": "https://www.gaokao.cn",
        "pragma": "no-cache",
        "referer": "https://www.gaokao.cn/",
        "sec-ch-ua": "\"Google Chrome\";v=\"119\", \"Chromium\";v=\"119\", \"Not?A_Brand\";v=\"24\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\"",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "cross-site",
        "user-agent": UserAgent().random,
    }
    url = "https://api.zjzw.cn/web/api/"

    # time.sleep(1)
    params = {
        "local_batch_id": f"{batch_id}",
        "local_province_id": f"{local_id}",
        "local_type_id": f"{type_id}",
        "page": f"{page}",
        "school_id": f"{school_id}",
        "size": "10",
        "special_group": "",
        "uri": "apidata/api/gkv3/plan/school",
        "year": f"{time_str}",
        "signsafe": "6a11b92fe5e62107fdcbcab79bafc805"  # 替换成你自己登录的结果
    }
    data = {
        "local_batch_id": batch_id,
        "local_province_id": local_id,
        "local_type_id": type_id,
        "page": page,
        "school_id": school_id,
        "signsafe": "6a11b92fe5e62107fdcbcab79bafc805",  # 替换成你自己登录的结果
        "size": 10,
        "special_group": "",
        "uri": "apidata/api/gkv3/plan/school",
        "year": int(time_str)
    }
    data = json.dumps(data, separators=(',', ':'))

    # 使用代理访问，访问三次失败切换并删除该ip地址
    global response
    retry_count = 3
    proxy = get_proxy()
    while retry_count > 0:
        try:
            response = requests.post(url, headers=headers, params=params, data=data,
                                     proxies={"http": "http://{}".format(proxy)}).json()

            # 如果提示访问太过频繁，切换ip
            if response['code'] != '0000':
                print(response)
                print("deleted",proxy)
                delete_proxy
                proxy = get_proxy()
                print("new",proxy)
                print("change new id")
                headers.update({"user-agent": UserAgent().random})
                headers.update({"Proxy-Tunnel": str(random.randint(1, 10000))})
                continue
            

            print(proxy)
            result_list = []
            items = response.get('data', {}).get('item', [])
            for item in items:
                # print(item)
                info = {}
                info['省份'] = local_id
                info['学校'] = item['name']
                info['学校代码'] = item['school_id']
                info['年份'] = item['year']
                info['科目'] = item['local_type_name']
                info['类型'] = item['local_batch_name']
                info['专业名称'] = item['spname']
                info['计划招生'] = item['num']
                info['学制'] = item['length']
                info['学费'] = item['tuition']
                info["_id"] = myhash(json.dumps(info))
                # print(info)
                result_list.append(info)

            return result_list
        except Exception:
            retry_count -= 1

    # 删除代理池中代理
    if retry_count == 0:
        delete_proxy(proxy)



In [28]:
import pandas as pd
import time
import random

# 读取schoolid.txt文件中的所有school id
with open('schoolid.txt', 'r') as file:
    school_ids = file.read().splitlines()

# 定义包含索引的DataFrame
index_df = pd.DataFrame({
    'index': [
        6, 36, 37, 86, 1527, 15, 12, 10, 14, 8, 45,
        44, 113, 1534, 46, 51, 111, 7, 38, 1939,
        1176, 1564, 1570, 98, 52, 90, 1137, 13, 1571, 69,11, 48, 16, 54, 100, 119, 47, 2726, 1561, 2929, 2734, 2741, 2642, 2667, 2583, 2685, 1565, 9, 1942
        ,2777, 2915, 2668, 2949, 2674, 1535, 1536, 2935, 2699, 2669, 17
    ]
})

for school_id in school_ids:
    # 根据school_id抓取数据
    df = extract_batch_data(int(school_id))
    print(df)

    # 检查是否所有数字都包含在索引中
    unique_batch = df['batch'].unique()
    not_in_index = [batch for batch in unique_batch if batch not in index_df['index'].values]
    if not_in_index:
        print("不包含在index里的数字:", not_in_index)
        break
    else:
        print("safe")

    selected_rows = df[df['batch'].isin([6, 36, 37, 86, 1527, 15, 90, 69, 11, 1535, 1536, 2935, 2699])]

    # 将筛选结果存储为新的DataFrame
    new_df = pd.DataFrame(selected_rows)

    # 打印新的DataFrame
    print(new_df)

    result_df = pd.DataFrame(columns=['省份', '学校', '学校代码', '年份', '科目', '类型', '专业名称', '计划招生', '学制', '学费', '_id'])
    start_time = time.time()

    for index, row in new_df.iterrows():
        province = row['province']
        year = row['year']
        data_type = row['data_type']
        batch = row['batch']

        print(f"Province: {province}, Year: {year}, Type: {data_type}, Batch: {batch}")

        for page in range(1, 6):
            result = get_data(int(school_id), year, province, batch, data_type, page)
            print(result)

            if not result:
                break

            if result:
                result_df = pd.concat([result_df, pd.DataFrame(result)])
                time.sleep(random.uniform(1, 2))

        # 将结果存储为CSV文件，文件名为学校名
        if not result_df.empty:
            result_df.to_csv(f"{result_df['学校'].values[0]}.csv", index=False)

    end_time = time.time()
    minutes, seconds = divmod(end_time - start_time, 60)
    print(f"运行时间：{round(minutes)} 分钟 {round(seconds)} 秒")

    province  year data_type batch
0         36  2023         1     8
1         36  2023         2     8
2         36  2022         1     8
3         36  2022         2     8
4         36  2021         1     8
..       ...   ...       ...   ...
162       53  2019         2  1534
163       53  2018         1     8
164       53  2018         2     8
165       65  2018         1    10
166       65  2018         2    10

[167 rows x 4 columns]
safe
    province  year data_type batch
128       50  2021      2073    37
130       50  2021      2074    37
Province: 50, Year: 2021, Type: 2073, Batch: 37
{'anonymous': '', 'check_count': 11, 'fail_count': 0, 'https': True, 'last_status': True, 'last_time': '2024-01-02 12:17:45', 'proxy': '134.35.133.189:8080', 'region': '也门 也门  yemen.net.ye', 'source': 'freeProxy11'}
[{'省份': '50', '学校': '重庆第二师范学院', '学校代码': '1489', '年份': '2021', '科目': '物理类', '类型': '本科提前批B段', '专业名称': '小学教育（全科教师定向潼南就业）（师范类。按重庆市相关文件执行，招收热爱祖国、热爱教师职业、品行良好、遵纪守法、志愿从事农村小学教育事业的重庆籍考生，毕业后在签约